The following notebook uses the NBA_API(https://github.com/swar/nba_api) to access game statistics and construct an ELO based model for Power Rankings inspired by Adam Silver's ELO model on https://fivethirtyeight.com/

In [1]:
from nba_api.stats.static import teams
import pandas as pd

In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

In [3]:
gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00')  

In [4]:
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42019,1610612743,DEN,Denver Nuggets,0041900234,2020-09-09,DEN vs. LAC,L,240,85,...,0.846,5,33,38,19,4,5,14,25,-11.0
1,42019,1610612761,TOR,Toronto Raptors,0041900216,2020-09-09,TOR @ BOS,W,289,125,...,0.720,8,38,46,22,10,8,12,26,3.0
2,42019,1610612738,BOS,Boston Celtics,0041900216,2020-09-09,BOS vs. TOR,L,290,122,...,0.833,14,44,58,32,5,4,15,25,-3.0
3,42019,1610612746,LAC,LA Clippers,0041900234,2020-09-09,LAC @ DEN,W,242,96,...,0.733,9,32,41,19,7,5,9,26,11.0
4,42019,1610612747,LAL,Los Angeles Lakers,0041900223,2020-09-08,LAL @ HOU,W,241,112,...,0.696,9,34,43,25,6,6,13,15,10.0


In [5]:
games['SEASON_ID'].unique()

array(['42019', '52019', '22019', '12019', '32019', '42018', '22018',
       '32018', '12018', '42017', '22017', '32017', '12017', '42016',
       '22016', '32016', '12016', '42015', '22015', '32015', '12015',
       '42014', '22014', '32014', '12014', '42013', '22013', '32013',
       '12013', '42012', '22012', '32012', '12012', '42011', '22011',
       '32011', '12011', '42010', '22010', '32010', '12010', '42009',
       '22009', '32009'], dtype=object)

In [6]:
elo_initial=1300

In [7]:
nba_games=games

In [8]:
nba_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42019,1610612743,DEN,Denver Nuggets,0041900234,2020-09-09,DEN vs. LAC,L,240,85,...,0.846,5,33,38,19,4,5,14,25,-11.0
1,42019,1610612761,TOR,Toronto Raptors,0041900216,2020-09-09,TOR @ BOS,W,289,125,...,0.720,8,38,46,22,10,8,12,26,3.0
2,42019,1610612738,BOS,Boston Celtics,0041900216,2020-09-09,BOS vs. TOR,L,290,122,...,0.833,14,44,58,32,5,4,15,25,-3.0
3,42019,1610612746,LAC,LA Clippers,0041900234,2020-09-09,LAC @ DEN,W,242,96,...,0.733,9,32,41,19,7,5,9,26,11.0
4,42019,1610612747,LAL,Los Angeles Lakers,0041900223,2020-09-08,LAL @ HOU,W,241,112,...,0.696,9,34,43,25,6,6,13,15,10.0


In [9]:
nba_games.shape

(30000, 28)

In [10]:
nba_games['elo_i']=1300
nba_games['elo_n']=1300
nba_games['opp_elo_i']=1300
nba_games['opp_elo_n']=1300

In [11]:
nba_games.sort_index(inplace=True,ascending=False)

In [13]:
nba_games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'elo_i', 'elo_n',
       'opp_elo_i', 'opp_elo_n'],
      dtype='object')

In [14]:
nba_games['game_order']=29999-nba_games.index

In [15]:
nba_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,STL,BLK,TOV,PF,PLUS_MINUS,elo_i,elo_n,opp_elo_i,opp_elo_n,game_order
29999,22009,1610612752,NYK,New York Knicks,0020900019,2009-10-30,NYK @ CHA,L,290,100,...,5,7,17,28,-2.0,1300,1300,1300,1300,0
29998,22009,1610612761,TOR,Toronto Raptors,0020900025,2009-10-30,TOR @ MEM,L,240,107,...,7,5,14,29,-8.0,1300,1300,1300,1300,1
29997,22009,1610612765,DET,Detroit Pistons,0020900023,2009-10-30,DET vs. OKC,L,241,83,...,6,5,15,27,-8.0,1300,1300,1300,1300,2
29996,22009,1610612766,CHA,Charlotte Bobcats,0020900019,2009-10-30,CHA vs. NYK,W,289,102,...,8,6,15,19,2.0,1300,1300,1300,1300,3
29995,22009,1610612760,OKC,Oklahoma City Thunder,0020900023,2009-10-30,OKC @ DET,W,240,91,...,6,5,11,28,8.0,1300,1300,1300,1300,4


In [16]:
temp=nba_games

In [17]:
nba_games=nba_games[nba_games['MATCHUP'].str.contains('vs')]

In [18]:
nba_games = nba_games.drop_duplicates(subset='GAME_ID', keep="first")

In [19]:
nba_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,STL,BLK,TOV,PF,PLUS_MINUS,elo_i,elo_n,opp_elo_i,opp_elo_n,game_order
29997,22009,1610612765,DET,Detroit Pistons,0020900023,2009-10-30,DET vs. OKC,L,241,83,...,6,5,15,27,-8.0,1300,1300,1300,1300,2
29996,22009,1610612766,CHA,Charlotte Bobcats,0020900019,2009-10-30,CHA vs. NYK,W,289,102,...,8,6,15,19,2.0,1300,1300,1300,1300,3
29994,22009,1610612738,BOS,Boston Celtics,0020900022,2009-10-30,BOS vs. CHI,W,239,118,...,7,5,16,22,28.0,1300,1300,1300,1300,5
29993,22009,1610612755,PHI,Philadelphia 76ers,0020900020,2009-10-30,PHI vs. MIL,W,241,99,...,12,7,19,22,13.0,1300,1300,1300,1300,6
29992,22009,1610612756,PHX,Phoenix Suns,0020900030,2009-10-30,PHX vs. GSW,W,240,123,...,7,7,23,25,22.0,1300,1300,1300,1300,7


In [20]:
nba_games['SEASON']=nba_games['SEASON_ID'].apply(lambda x: x[1:])

In [21]:
nba_games['SEASON']=nba_games['SEASON'].astype(int).apply(lambda x: "%s-%s"%(x,x+1))

In [22]:
nba_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,BLK,TOV,PF,PLUS_MINUS,elo_i,elo_n,opp_elo_i,opp_elo_n,game_order,SEASON
29997,22009,1610612765,DET,Detroit Pistons,0020900023,2009-10-30,DET vs. OKC,L,241,83,...,5,15,27,-8.0,1300,1300,1300,1300,2,2009-2010
29996,22009,1610612766,CHA,Charlotte Bobcats,0020900019,2009-10-30,CHA vs. NYK,W,289,102,...,6,15,19,2.0,1300,1300,1300,1300,3,2009-2010
29994,22009,1610612738,BOS,Boston Celtics,0020900022,2009-10-30,BOS vs. CHI,W,239,118,...,5,16,22,28.0,1300,1300,1300,1300,5,2009-2010
29993,22009,1610612755,PHI,Philadelphia 76ers,0020900020,2009-10-30,PHI vs. MIL,W,241,99,...,7,19,22,13.0,1300,1300,1300,1300,6,2009-2010
29992,22009,1610612756,PHX,Phoenix Suns,0020900030,2009-10-30,PHX vs. GSW,W,240,123,...,7,23,25,22.0,1300,1300,1300,1300,7,2009-2010


In [23]:
nba_games.shape

(14998, 34)

In [24]:
len(nba_games['TEAM_ABBREVIATION'].unique())

53

In [26]:
nba_games['AWAY']=nba_games['MATCHUP'].apply(lambda x: x[-3:])

In [27]:
nba_games['AWAY_PTS']=nba_games['PTS']-nba_games['PLUS_MINUS']

In [28]:
nba_games['year_id']=nba_games['SEASON_ID'].apply(lambda x: x[1:]).astype(int)

In [29]:
l=nba_games.shape[0]
ind=range(0,l)
nba_games.index=ind

In [30]:
nba_games['game_order']=nba_games.index

In [31]:
nba_games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PLUS_MINUS,elo_i,elo_n,opp_elo_i,opp_elo_n,game_order,SEASON,AWAY,AWAY_PTS,year_id
14993,42019,1610612761,TOR,Toronto Raptors,0041900215,2020-09-07,TOR vs. BOS,L,241,89,...,-22.0,1300,1300,1300,1300,14993,2019-2020,BOS,111.0,2019
14994,42019,1610612745,HOU,Houston Rockets,0041900223,2020-09-08,HOU vs. LAL,L,240,102,...,-10.0,1300,1300,1300,1300,14994,2019-2020,LAL,112.0,2019
14995,42019,1610612749,MIL,Milwaukee Bucks,0041900205,2020-09-08,MIL vs. MIA,L,241,94,...,-9.0,1300,1300,1300,1300,14995,2019-2020,MIA,103.0,2019
14996,42019,1610612738,BOS,Boston Celtics,0041900216,2020-09-09,BOS vs. TOR,L,290,122,...,-3.0,1300,1300,1300,1300,14996,2019-2020,TOR,125.0,2019
14997,42019,1610612743,DEN,Denver Nuggets,0041900234,2020-09-09,DEN vs. LAC,L,240,85,...,-11.0,1300,1300,1300,1300,14997,2019-2020,LAC,96.0,2019


In [32]:
nba_games['year_id']

0        2009
1        2009
2        2009
3        2009
4        2009
         ... 
14993    2019
14994    2019
14995    2019
14996    2019
14997    2019
Name: year_id, Length: 14998, dtype: int64

In [33]:
nba_games['TEAM_ABBREVIATION'] = nba_games['TEAM_ABBREVIATION'].replace(['NOH'],'NOP')
nba_games['TEAM_ABBREVIATION'] = nba_games['TEAM_ABBREVIATION'].replace(['NJN'],'BKN')
nba_games['AWAY'] = nba_games['AWAY'].replace(['NOH'],'NOP')
nba_games['AWAY'] = nba_games['AWAY'].replace(['NJN'],'BKN')

In [34]:
nba_teamslist=['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS']

In [35]:
nba_games=nba_games[nba_games['TEAM_ABBREVIATION'].isin(nba_teamslist)]

In [36]:
nba_games=nba_games[nba_games['AWAY'].isin(nba_teamslist)]

In [37]:
l1=list(nba_games['TEAM_ABBREVIATION'].unique())
l2=list(nba_games['AWAY'].unique())
for i in l1:
    if i not in l2:
        l2.append(i)
l2

['OKC',
 'NYK',
 'CHI',
 'MIL',
 'GSW',
 'CLE',
 'MIA',
 'SAC',
 'TOR',
 'PHI',
 'DAL',
 'CHA',
 'POR',
 'BKN',
 'DET',
 'ATL',
 'NOP',
 'MEM',
 'ORL',
 'MIN',
 'HOU',
 'WAS',
 'DEN',
 'UTA',
 'PHX',
 'BOS',
 'LAL',
 'IND',
 'SAS',
 'LAC']

In [38]:
len(l2)

30

In [39]:
from collections import defaultdict
import numpy as np

def silverK(MOV, elo_diff):
    K_0=20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return K_0*multiplier,K_0*multiplier

def silverS(home_score, away_score):
    S_home,S_away=0,0
    if home_score>away_score:
        S_home=1
    elif away_score>home_score:
        S_away=1
    else:
        S_home,S_away=.5,.5
    return S_home,S_away

def silver_elo_update(home_score, away_score, home_rating, away_rating):
    HOME_AD=100.
    home_rating+=HOME_AD
    E_home = elo_prediction(home_rating,away_rating)
    E_away=1-E_home
    elo_diff=home_rating-away_rating
    MOV=home_score-away_score
    
    S_home,S_away = silverS(home_score,away_score)
    if S_home>0:
        K_home,K_away =  silverK(MOV,elo_diff)
    else:
        K_home,K_away =  silverK(MOV,elo_diff)
        
    return K_home*(S_home-E_home),K_away*(S_away-E_away)

def elo_prediction(home_rating,away_rating):
    E_home = 1./(1 + 10 ** ((away_rating - home_rating) / (400.)))
    return E_home

def score_prediction(home_rating,away_rating):
    return (home_rating-away_rating)/28.

class HeadToHeadModel(object):
    def __init__(self, events, update_function, prediction_function=None):
        self.update_function=update_function
        self.events=events
        self.ratings={}
        self.prediction_function = prediction_function
        self.predictions = []
        self.curr_season=defaultdict(lambda: self.events[0][1]['year_id'])

    def train(self):
        for idx, event in self.events:
            new_year=event['year_id']
            label_i=event['TEAM_ABBREVIATION']
            label_j=event['AWAY']
            if self.ratings.get(label_i,False)==False:
                self.ratings[label_i]=elo_lookup(label_i,event['game_order'])
            if self.ratings.get(label_j,False)==False:
                self.ratings[label_j]=elo_lookup(label_j,event['game_order'])
                
            if self.curr_season[label_i]!=new_year:
                self.curr_season[label_i]=new_year
                self.ratings[label_i]=self.ratings[label_i]*.75+1505.*.25
            elif self.curr_season[label_j]!=new_year:
                self.curr_season[label_j]=new_year
                self.ratings[label_j]=self.ratings[label_j]*.75+1505.*.25
            #todo change below to just use event
            update=self.update_function(event['PTS'],event['AWAY_PTS'], self.ratings[label_i], self.ratings[label_j])
            self.ratings[label_i]+=update[0]
            self.ratings[label_j]+=update[1]
            

    def power_rankings(self):
        from operator import itemgetter
        power_rankings = sorted(self.ratings.items(), key=itemgetter(1), reverse=True)
        power = []
        for i, x in enumerate(power_rankings):
            power.append((i + 1, x))
        return power
    
STARTING_LOC=-1
def elo_lookup(fran_id,gameorder):
    return 1300
m=HeadToHeadModel(list(nba_games[nba_games['game_order']>STARTING_LOC][:].iterrows()), silver_elo_update, elo_prediction)
m.train()
m.power_rankings()

[(1, ('BOS', 1682.0177436258589)),
 (2, ('TOR', 1670.3861018436994)),
 (3, ('LAC', 1654.2608046403654)),
 (4, ('LAL', 1619.894780369134)),
 (5, ('MIA', 1616.7077499167303)),
 (6, ('MIL', 1607.5640138097956)),
 (7, ('PHX', 1577.9019595589623)),
 (8, ('HOU', 1572.824460373603)),
 (9, ('OKC', 1568.5196568437084)),
 (10, ('NOP', 1543.6854405225454)),
 (11, ('PHI', 1538.6592771935898)),
 (12, ('DAL', 1535.771094160252)),
 (13, ('MEM', 1527.6898351771279)),
 (14, ('UTA', 1527.6276878017588)),
 (15, ('IND', 1521.5420812560021)),
 (16, ('DEN', 1516.0306414784307)),
 (17, ('SAS', 1515.3395722883192)),
 (18, ('SAC', 1500.930796517313)),
 (19, ('POR', 1484.623238182907)),
 (20, ('ORL', 1473.0288132794965)),
 (21, ('BKN', 1454.6367856550664)),
 (22, ('CHA', 1384.4418739298274)),
 (23, ('WAS', 1357.9546612004974)),
 (24, ('GSW', 1341.3624123234235)),
 (25, ('MIN', 1340.6743102017847)),
 (26, ('ATL', 1336.7605586353893)),
 (27, ('CHI', 1331.7427908197926)),
 (28, ('DET', 1327.2866369726387)),
 (29, 

2019-20 NBA Season as of 10th September-2020